In [1]:
# Allows imports from other packages in the project
import sys
import os
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(project_root)

In [2]:
import pandas as pd

In [3]:
stocks_by_sector_df = pd.read_csv("../data/stocks_by_sector.csv")
stocks_by_sector_df

,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology
0,T,AMZN,WMT,MRO,BAC,PFE,CSX,AAPL
1,GOOGL,F,KO,KMI,WFC,MRK,GE,AMD
2,GOOG,GM,KR,XOM,C,GILD,DAL,NVDA
3,CMCSA,EBAY,PG,HAL,RF,BMY,AAL,MU
4,VZ,SBUX,MDLZ,WMB,JPM,BSX,CPRT,MSFT
5,NFLX,NKE,MO,COP,KEY,ABT,LUV,INTC
6,DIS,M,MNST,SLB,MS,CVS,FAST,CSCO
7,IPG,MGM,COTY,DVN,HBAN,ABBV,CAT,HPE
8,EA,TJX,WBA,CVX,SCHW,JNJ,JCI,ORCL
9,NWSA,TGT,PM,MPC,SYF,MDT,UAL,AMAT


## Get the data and the indicators

Stock data is collected with `yfinance` for dates `2016-01-02 - 2019-01-01` (3 years). Certain indicators are then calculated for analysis. These indicators are:

- Relative Strength Index (RSI)
- Stochastic Oscillator
- Williams %R
- Moving Average Convergence Divergence (MACD)
- Price Rate Of Change
- On Balance Volume

A new dataframe with columns `['Close', 'RSI', 'k_percent', 'r_percent', 'MACD', 'MACD_EMA9', 'Price Rate Of Change', 'On Balance Volume']` is then created to reprent a single stock.

Each of these dataframes are then concatenated to obtain the resulting samples dataframe of shape `(T, NM)`, where `T` is the number of time steps, `N` the number of assets, and `M` the number of indicators for each asset.

Asset names are standardized (e.g. asset_1) for tensorization.

Note that the labels are set according to the price change `n` days-out (set to be 9 below).

In [4]:
from grtel.indicators import get_indicators

In [5]:
samples = []
labels = []
for sector in stocks_by_sector_df.columns:
    for i, ticker in enumerate(stocks_by_sector_df[sector]):
        asset_name = f'asset_{i + 1}'

        stock_data_and_indicators = get_indicators(ticker)

        X_i = stock_data_and_indicators[['Close', 'RSI', 'k_percent', 'r_percent', 'MACD', 'MACD_EMA9', 'Price Rate Of Change', 'On Balance Volume']].copy()
        num_indicators = len(X_i.columns)
        X_i.columns = [ [sector] * num_indicators, [asset_name] * num_indicators, X_i.columns]

        y_i = stock_data_and_indicators['Prediction'].copy()
        y_i.name = ticker

        samples.append(X_i)
        labels.append(y_i)

samples_df = pd.concat(samples, axis=1)
samples_df.columns.names = ['Sector', 'Asset', 'Metrics']

labels_df = pd.concat(labels, axis=1)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [6]:
samples_df

Sector     Communication Services                                             \
Asset                     asset_1                                              
Metrics                     Close        RSI  k_percent  r_percent      MACD   
Date                                                                           
2016-01-22              26.540785  66.984964  97.740045  -2.259955  0.069637   
2016-01-25              26.435045  63.461389  78.712858 -21.287142  0.093537   
2016-01-26              26.737160  68.861320  92.129624  -7.870376  0.128820   
2016-01-27              26.797583  69.888271  86.610891 -13.389109  0.157869   
2016-01-28              26.835346  70.587786  88.702900 -11.297100  0.180559   
...                           ...        ...        ...        ...       ...   
2018-12-11              22.432024  36.639805  18.662003 -81.337997 -0.177166   
2018-12-12              22.779455  46.469092  34.859142 -65.140858 -0.175076   
2018-12-13              22.590633  42.349233  26.056326 -73.943674 -0.186507   
2018-12-14              22.824774  48.839002  36.971882 -63.028118 -0.174659   
2018-12-17              22.552870  42.437767  24.295798 -75.704202 -0.185076   

Sector                                                                  \
Asset                                                          asset_2   
Metrics    MACD_EMA9 Price Rate Of Change On Balance Volume      Close   
Date                                                                     
2016-01-22  0.016062             0.047704          14024206  37.272999   
2016-01-25  0.032122             0.030928         -26045860  36.681000   
2016-01-26  0.052022             0.044248          29134620  36.689499   
2016-01-27  0.073679             0.051571          95682832  35.879002   
2016-01-28  0.095447             0.035860         130350978  37.415001   
...              ...                  ...               ...        ...   
2018-12-11 -0.180185            -0.024310          35111028  53.082500   
2018-12-12 -0.179164            -0.018868          89267527  53.686501   
2018-12-13 -0.180632            -0.021590          24262040  53.676998   
2018-12-14 -0.179438            -0.032650          80851654  52.585499   
2018-12-17 -0.180565            -0.058341          22566923  51.282501   

Sector                 ... Information Technology                    \
Asset                  ...                asset_9                     
Metrics           RSI  ...   Price Rate Of Change On Balance Volume   
Date                   ...                                            
2016-01-22  58.189602  ...               0.004329         -16351600   
2016-01-25  49.067155  ...              -0.017172         -33145200   
2016-01-26  49.199084  ...              -0.003958         -13416300   
2016-01-27  38.286812  ...               0.024354         -33983900   
2016-01-28  58.442415  ...               0.015522         -18511100   
...               ...  ...                    ...               ...   
2018-12-11  47.241232  ...              -0.020658         682301800   
2018-12-12  52.226371  ...              -0.018054         703705800   
2018-12-13  52.136943  ...              -0.013988         680410400   
2018-12-14  42.493301  ...              -0.044299         658976100   
2018-12-17  33.865169  ...              -0.082280         628535000   

Sector                                                                      \
Asset        asset_10                                                        
Metrics         Close        RSI  k_percent  r_percent      MACD MACD_EMA9   
Date                                                                         
2016-01-22  17.049999  51.119297  43.165445 -56.834555 -0.097708 -0.124555   
2016-01-25  16.910000  47.801520  38.129494 -61.870506 -0.078060 -0.114917   
2016-01-26  17.350000  57.746426  63.025232 -36.974768 -0.035484 -0.098570   
2016-01-27  17.090000  51.107526  70.454523 -29.545477 -0.018663 -0.082221  

In [7]:
labels_df

,T,GOOGL,GOOG,CMCSA,VZ,NFLX,DIS,IPG,EA,NWSA,...,AAPL,AMD,NVDA,MU,MSFT,INTC,CSCO,HPE,ORCL,AMAT
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-22,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
2016-01-25,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2016-01-26,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2016-01-27,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2016-01-28,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
samples_df.to_csv('../data/samples_sector.csv')
labels_df.to_csv('../data/labels_sector.csv')